<a href="https://colab.research.google.com/github/JoaquinV13/UFV-Visualizacion/blob/main/Clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
pip install plotly_express

In [86]:
import plotly_express as px
import pandas as pd
import numpy as np
from google.colab import drive

In [87]:
drive.mount('/content/drive/')
data = pd.read_csv('/content/drive/MyDrive/Colab/datos_ejercicio_ventas.csv',sep=';')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [88]:
# Visualizamos los datos.
data.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,"8,8502E+15"
1,Spain,Pepsi Regular (L3),2023,12,AI_forecast,AI_P05F,2023.0,"1,34268E+15"
2,Spain,7up Free (L3),2023,8,AI_forecast,AI_P08F,2023.0,"8,98326E+15"
3,Spain,Pepsi Regular (L3),2023,8,AI_forecast,AI_PF,2023.0,"1,6476E+15"
4,Spain,Pepsi Max (L3),2023,10,AI_forecast,AI_P07F,2023.0,"1,41119E+16"


In [89]:
# Tipos de datos.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18666 entries, 0 to 18665
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   COUNTRY        18666 non-null  object 
 1   SUBBRAND       18666 non-null  object 
 2   YEAR           18666 non-null  int64  
 3   MONTH          18666 non-null  int64  
 4   SCENARIO       18666 non-null  object 
 5   FORECAST       17766 non-null  object 
 6   FORECAST_YEAR  17766 non-null  float64
 7   AMOUNT         18666 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 1.1+ MB


In [90]:
# Número de 'Actuals' y 'AI_forecast'.
actuals = (sum(data['SCENARIO']=='actual'))
forecast = (sum(data['SCENARIO']=='AI_forecast'))
print('Número de Actuals:',actuals)
print('Número de Forecasts:',forecast)

Número de Actuals: 900
Número de Forecasts: 17766


In [91]:
# Número de regustros de cada marca por país.
tabla_pivot = data.pivot_table(index='COUNTRY', columns='SUBBRAND', aggfunc='size', fill_value=0)

# Agregar una columna con el total de registros por país.
tabla_pivot['Total'] = tabla_pivot.sum(axis=1)
print(tabla_pivot)

SUBBRAND       7up (L3)  7up Free (L3)  Lipton (L3)  Mountain Dew (L3)  \
COUNTRY                                                                  
Czech               236              7          236                236   
Denmark             216            236            0                236   
Great Britain       794            758          830                542   
Hungary             268            236          452                236   
Italy               236              0            0                  0   
Netherlands         236            236          452                216   
Norway              236            236          252                236   
Portugal            236            236          235                  0   
Spain               236            398          236                  0   

SUBBRAND       Pepsi Max (L3)  Pepsi Regular (L3)  Total  
COUNTRY                                                   
Czech                     236                 452   1403  
Denmark 

In [92]:
# Todos los datos del dataset tiene el mismo horizonte de previsión. Calculamos
# el de uno, sabemos el de todos.
data_1 = data[(data['FORECAST_YEAR']==2023) & (data['FORECAST']=='AI_P02F') & (data['COUNTRY']=='Portugal') & (data['SCENARIO']=='AI_forecast') & (data['SUBBRAND']=='Lipton (L3)')]
print(len(data_1))

18


In [93]:
# Comprobamos la cantidad de forecasts distintos que hay. Contando las diferentes
# fechas de inicio de un forecast.

data[(data['SCENARIO']=='AI_forecast')].loc[:, ['FORECAST_YEAR', 'FORECAST']].drop_duplicates().shape[0]

12

Vemos que la cantidad de forecast distintos es 12 pero, ¿esto es igual para todos los productos de cada país?

In [94]:
# Comprobamos que para cada producto de cada país se han hecho 12 estudios
# predicitivos teniendo en cuenta la fecha de inicio de la predicción.

Pais_producto = data[(data['SCENARIO']=='AI_forecast')].groupby(['COUNTRY','SUBBRAND'])

# Contar los valores únicos de la combinación de 'Year' y 'caca'
resultado = Pais_producto.apply(lambda x: x[['FORECAST_YEAR', 'FORECAST']].drop_duplicates().shape[0], include_groups=False).reset_index(name='Estudios')

print(resultado)

          COUNTRY            SUBBRAND  Estudios
0           Czech            7up (L3)        12
1           Czech         Lipton (L3)        12
2           Czech   Mountain Dew (L3)        12
3           Czech      Pepsi Max (L3)        12
4           Czech  Pepsi Regular (L3)        12
5         Denmark            7up (L3)        12
6         Denmark       7up Free (L3)        12
7         Denmark   Mountain Dew (L3)        12
8         Denmark      Pepsi Max (L3)        12
9         Denmark  Pepsi Regular (L3)        12
10  Great Britain            7up (L3)        12
11  Great Britain       7up Free (L3)        12
12  Great Britain         Lipton (L3)        12
13  Great Britain   Mountain Dew (L3)        12
14  Great Britain      Pepsi Max (L3)        12
15  Great Britain  Pepsi Regular (L3)        12
16        Hungary            7up (L3)        12
17        Hungary       7up Free (L3)        12
18        Hungary         Lipton (L3)        12
19        Hungary   Mountain Dew (L3)   

Efectivamente, para cada producto de cada país se han hecho 12 estudios de predicción distintos.

In [95]:
# Convertimos la columna AMOUNT en formato numérico.
data['AMOUNT'] = data['AMOUNT'].str.replace(',', '', regex=False)
data['AMOUNT'] = pd.to_numeric(data['AMOUNT'])

In [96]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18666 entries, 0 to 18665
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   COUNTRY        18666 non-null  object 
 1   SUBBRAND       18666 non-null  object 
 2   YEAR           18666 non-null  int64  
 3   MONTH          18666 non-null  int64  
 4   SCENARIO       18666 non-null  object 
 5   FORECAST       17766 non-null  object 
 6   FORECAST_YEAR  17766 non-null  float64
 7   AMOUNT         18666 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 1.1+ MB


In [97]:
# Sumamos la cantidad de cada producto del país.
datos = data[(data['SCENARIO'] == 'actual')].groupby(['COUNTRY', 'SUBBRAND'])['AMOUNT'].sum().reset_index()
print(datos)

          COUNTRY            SUBBRAND        AMOUNT
0           Czech            7up (L3)  1.919635e+22
1           Czech       7up Free (L3)  2.740001e+21
2           Czech         Lipton (L3)  1.933554e+22
3           Czech   Mountain Dew (L3)  1.460128e+22
4           Czech      Pepsi Max (L3)  3.060774e+22
5           Czech  Pepsi Regular (L3)  1.135877e+22
6         Denmark       7up Free (L3)  3.172219e+22
7         Denmark   Mountain Dew (L3)  2.603437e+22
8         Denmark      Pepsi Max (L3)  2.361710e+22
9         Denmark  Pepsi Regular (L3)  2.273068e+22
10  Great Britain            7up (L3)  1.735373e+22
11  Great Britain       7up Free (L3)  1.398643e+22
12  Great Britain         Lipton (L3)  8.391507e+21
13  Great Britain   Mountain Dew (L3)  2.043746e+22
14  Great Britain      Pepsi Max (L3)  1.856443e+29
15  Great Britain  Pepsi Regular (L3)  2.045068e+22
16        Hungary            7up (L3)  9.103885e+21
17        Hungary       7up Free (L3)  1.018670e+22
18        Hu

In [98]:
# Normalizamos las cantidades de los productos diviendo entre el total de
# de cantidad vendida en el país.
country_totals = datos.groupby('COUNTRY')['AMOUNT'].sum().reset_index()
country_totals = country_totals.rename(columns={'AMOUNT': 'COUNTRY_TOTAL'})

datos_N = pd.merge(datos, country_totals, on='COUNTRY', how='left')
datos_N['AMOUNT_N'] = datos_N['AMOUNT'] / datos_N['COUNTRY_TOTAL']

# Eliminamos columnas innecesarias.
datos_N.drop(columns=['AMOUNT', 'COUNTRY_TOTAL'], inplace=True)

print(datos_N)

          COUNTRY            SUBBRAND      AMOUNT_N
0           Czech            7up (L3)  1.962021e-01
1           Czech       7up Free (L3)  2.800500e-02
2           Czech         Lipton (L3)  1.976247e-01
3           Czech   Mountain Dew (L3)  1.492368e-01
4           Czech      Pepsi Max (L3)  3.128357e-01
5           Czech  Pepsi Regular (L3)  1.160958e-01
6         Denmark       7up Free (L3)  3.047154e-01
7         Denmark   Mountain Dew (L3)  2.500796e-01
8         Denmark      Pepsi Max (L3)  2.268599e-01
9         Denmark  Pepsi Regular (L3)  2.183452e-01
10  Great Britain            7up (L3)  9.347835e-08
11  Great Britain       7up Free (L3)  7.533988e-08
12  Great Britain         Lipton (L3)  4.520204e-08
13  Great Britain   Mountain Dew (L3)  1.100893e-07
14  Great Britain      Pepsi Max (L3)  9.999996e-01
15  Great Britain  Pepsi Regular (L3)  1.101605e-07
16        Hungary            7up (L3)  7.809597e-02
17        Hungary       7up Free (L3)  8.738473e-02
18        Hu

# Distribución de ventas realizadas en cada país.

In [99]:
fig = px.histogram(datos_N, x='COUNTRY', y='AMOUNT_N', color='SUBBRAND',
    title="Distribución de ventas realizadas por país",
    template="plotly_white",
    color_discrete_sequence=px.colors.qualitative.Prism, # Color pastel.
    height=700, # Modificamos la altura para que se vean bien las porporciones.
    width=1100)

fig.update_layout(xaxis_title="País",  yaxis_title="Proporción de las ventas",
    legend_title="Marca",  font=dict(family="Arial", size=12),
    bargap=0.4) # Ajustamos el espacio entre barras.

fig.show()

In [113]:
data['copia_Marca'] = data['SUBBRAND']
Marca = data[data['SCENARIO'] == 'actual'].groupby('copia_Marca').apply(lambda x: x.sort_values(['YEAR', 'MONTH']),
       include_groups=False).reset_index(drop=True)
Marca.drop(columns=['COUNTRY','SCENARIO','FORECAST', 'FORECAST_YEAR'], inplace=True)
data.drop(columns=['copia_Marca'], inplace=True)
print(Marca)

               SUBBRAND  YEAR  MONTH        AMOUNT
0              7up (L3)  2023      1  7.897200e+19
1              7up (L3)  2023      1  2.503220e+21
2              7up (L3)  2023      1  1.485700e+20
3              7up (L3)  2023      1  1.221620e+21
4              7up (L3)  2023      1  1.068160e+21
..                  ...   ...    ...           ...
895  Pepsi Regular (L3)  2024      8  2.132090e+20
896  Pepsi Regular (L3)  2024      8  9.817880e+19
897  Pepsi Regular (L3)  2024      8  2.035190e+20
898  Pepsi Regular (L3)  2024      8  3.761880e+19
899  Pepsi Regular (L3)  2024      8  9.852900e+18

[900 rows x 4 columns]


# Distriución de ventas según mes y año.